In [1]:
#импорт библиотек
import pandas as pd
import numpy as np
import textwrap
import os
import plotly.express as px 
import plotly.io as pio
from copy import deepcopy

# Часть 1

In [2]:
#считываем файл
data = pd.read_csv('Тестовое задание - tz_data.csv')

In [3]:
#общая информация о файле
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229 entries, 0 to 228
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area          228 non-null    object 
 1   cluster       228 non-null    float64
 2   cluster_name  228 non-null    object 
 3   keyword       228 non-null    object 
 4   good (1)      227 non-null    float64
 5   count         227 non-null    object 
 6   x             228 non-null    float64
 7   y             228 non-null    object 
dtypes: float64(3), object(5)
memory usage: 14.4+ KB


In [4]:
#первые 5 строк
data.head()

,area,cluster,cluster_name,keyword,good (1),count,x,y
0,eligibility,0.0,Кластер 0,several animated buried,1.0,1260,5.772342,12.564796257345005
1,eligibility,0.0,Кластер 0,singles unusual buyers,1.0,866,14.829280,7.8507285727125815
2,eligibility,0.0,Кластер 0,hawaiian directive,1.0,163,11.381856,3.8981370219558604
3,eligibility,0.0,Кластер 0,dynamics directly,1.0,1146,9.980149,6.281427914064545
4,eligibility,1.0,Кластер 1,decision surgeons montreal,1.0,823,3.283940,4.39674063521296


In [5]:
#смотрим пропуски
data.isna().sum()

area            1
cluster         1
cluster_name    1
keyword         1
good (1)        2
count           2
x               1
y               1
dtype: int64

### 1. В выходной таблице оставляем только нужные колонки

In [6]:
#удалим столбец
data.drop('good (1)', axis=1, inplace=True)

In [7]:
data[data['count'].isna()]

,area,cluster,cluster_name,keyword,count,x,y
178,housewives,2.0,Кластер 2,outstanding relations,NaN,1.289158,6.397514584626231
193,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#удалим полностью пустую строку
data.drop(labels=[193], inplace=True)

In [9]:
#заполним оставшийся пропуск заглушкой
data['count'].fillna(-1, inplace=True)

In [10]:
data.isna().sum()

area            0
cluster         0
cluster_name    0
keyword         0
count           0
x               0
y               0
dtype: int64

In [11]:
#смотрим уникальные значения в столбце 'count'
#data['count'].value_counts().index.tolist()

In [12]:
#найдем индексы с "мусорными" значениями в столбце 'count'
idx = data[(data['count']=='N\\A') | (data['count']=='-')].index.tolist()

In [13]:
#заменим их
data.loc[idx, 'count'] = -1

In [14]:
#изменим тип данных
data['cluster'] = data['cluster'].astype('int64')
data['count'] = data['count'].astype('int64')

In [15]:
idx_2 = data[data['y']=='0x414fe002'].index.tolist() #аналогично со столбцом 'y'

In [16]:
data.loc[idx_2, 'y'] = -1

In [17]:
data['y'] = data['y'].astype('float64')

### 2. Добавляем колонку color

In [18]:
#функция добавления цвета
#для каждой области цвета различных кластеров будут одинаковыми (но разными внутри области)
def define_color(value):
    if value == 0:
        return 'red'
    elif value == 1:
        return 'green'
    elif value == 2:
        return 'blue'
    elif value == 3:
        return 'orange'

In [19]:
data['color'] = data.cluster.apply(define_color)

In [20]:
data

,area,cluster,cluster_name,keyword,count,x,y,color
0,eligibility,0,Кластер 0,several animated buried,1260,5.772342,12.564796,red
1,eligibility,0,Кластер 0,singles unusual buyers,866,14.829280,7.850729,red
2,eligibility,0,Кластер 0,hawaiian directive,163,11.381856,3.898137,red
3,eligibility,0,Кластер 0,dynamics directly,1146,9.980149,6.281428,red
4,eligibility,1,Кластер 1,decision surgeons montreal,823,3.283940,4.396741,green
...,...,...,...,...,...,...,...,...
224,greetings,2,Кластер 2,disposition layout,279,10.971214,4.857810,blue
225,greetings,2,Кластер 2,sapphire grounds,335,1.160626,3.642820,blue
226,greetings,3,Кластер 3,entire ethical speakers,1782,7.985910,6.003699,orange
227,greetings,3,Кластер 3,courtesy textiles diameter,84,0.509490,4.151199,orange


### 3. Не должно быть дубликатов слов в одной и той же области (area), но словосочетание может повторяться из area в area


In [21]:
#уникальные значения каждой области
areas = data.area.value_counts().index.tolist()
areas

['eligibility',
 'twisted',
 'greetings',
 'capability',
 'available',
 'protein',
 'winner',
 'locator',
 'worlds',
 'ar\\vr',
 'lithuania',
 'personnel',
 'housewives',
 'dialog',
 'except']

In [22]:
#удаляем дубликаты внутри каждой области и создаем новую таблицу df

df = pd.DataFrame()
for area in areas:
    temp = data[data['area']==area].drop_duplicates(subset=['keyword'])
    df = pd.concat([df, temp], ignore_index=True)

In [23]:
#количество строк сократилось, значит, дубликаты были
df

,area,cluster,cluster_name,keyword,count,x,y,color
0,eligibility,0,Кластер 0,several animated buried,1260,5.772342,12.564796,red
1,eligibility,0,Кластер 0,singles unusual buyers,866,14.829280,7.850729,red
2,eligibility,0,Кластер 0,hawaiian directive,163,11.381856,3.898137,red
3,eligibility,0,Кластер 0,dynamics directly,1146,9.980149,6.281428,red
4,eligibility,1,Кластер 1,decision surgeons montreal,823,3.283940,4.396741,green
...,...,...,...,...,...,...,...,...
216,except,2,Кластер 2,justin thailand inspection,347,9.711229,10.178580,blue
217,except,2,Кластер 2,jacksonville tomatoes alberta,1968,6.760798,10.922626,blue
218,except,3,Кластер 3,missions introduced,987,8.042422,5.623459,orange
219,except,3,Кластер 3,subscribers member,778,10.046830,3.413063,orange


### Колонки должны называться именно так, как указано в п.1: area, cluster, cluster_name, keyword, x, y, count, color

In [24]:
#меняем местами столбцы
df = df.reindex(columns=['area', 'cluster', 'cluster_name', 'keyword',  'x', 'y', 'count', 'color'])
df

,area,cluster,cluster_name,keyword,x,y,count,color
0,eligibility,0,Кластер 0,several animated buried,5.772342,12.564796,1260,red
1,eligibility,0,Кластер 0,singles unusual buyers,14.829280,7.850729,866,red
2,eligibility,0,Кластер 0,hawaiian directive,11.381856,3.898137,163,red
3,eligibility,0,Кластер 0,dynamics directly,9.980149,6.281428,1146,red
4,eligibility,1,Кластер 1,decision surgeons montreal,3.283940,4.396741,823,green
...,...,...,...,...,...,...,...,...
216,except,2,Кластер 2,justin thailand inspection,9.711229,10.178580,347,blue
217,except,2,Кластер 2,jacksonville tomatoes alberta,6.760798,10.922626,1968,blue
218,except,3,Кластер 3,missions introduced,8.042422,5.623459,987,orange
219,except,3,Кластер 3,subscribers member,10.046830,3.413063,778,orange


### Сортировка должна происходить по колонкам area, cluster, cluster_name, count (по count значения сортируются в убывающем порядке, в остальных - по возрастающему).


In [25]:
#cначала отсортируем по 'area', 'cluster', 'cluster_name' в порядке возрастания
df.sort_values(by=['area', 'cluster', 'cluster_name'], inplace=True)

In [26]:
df.head(14)

,area,cluster,cluster_name,keyword,x,y,count,color
133,ar\vr,0,Кластер 0,written conflict fabulous,2.991167,7.106799,1443,red
134,ar\vr,0,Кластер 0,interfaces neutral,10.443533,13.809915,586,red
135,ar\vr,0,Кластер 0,reservations linking,10.195602,12.259496,751,red
136,ar\vr,0,Кластер 0,committees parallel,6.735260,3.613983,173,red
137,ar\vr,1,Кластер 1,postcards looked republic detector,10.474474,6.220012,1397,green
138,ar\vr,1,Кластер 1,filling volunteers academics,10.833920,8.652737,773,green
139,ar\vr,1,Кластер 1,michael tobacco,6.210880,12.721264,1007,green
140,ar\vr,1,Кластер 1,celtic automation,12.197275,4.723751,715,green
141,ar\vr,2,Кластер 2,bangkok mining fascinating,5.084247,13.480031,682,blue
142,ar\vr,2,Кластер 2,previously standing languages commands,14.668089,8.467136,889,blue


In [27]:
#уникальные значения областей в алфавитном порядке
sort_areas = df.area.value_counts().sort_index().index.tolist()
sort_areas

['ar\\vr',
 'available',
 'capability',
 'dialog',
 'eligibility',
 'except',
 'greetings',
 'housewives',
 'lithuania',
 'locator',
 'personnel',
 'protein',
 'twisted',
 'winner',
 'worlds']

In [28]:
#создаем новую таблицу с сортировкой по столбцу 'count'
sort_df = pd.DataFrame()
for area in sort_areas:
    for cluster in df[df['area']==area].cluster.value_counts().index.tolist():
        sorted = df[(df['area']==area) & (df['cluster']==cluster)].sort_values(by='count', ascending=False)
        sort_df = pd.concat([sort_df, sorted], )

In [29]:
sort_df.head(14)

,area,cluster,cluster_name,keyword,x,y,count,color
133,ar\vr,0,Кластер 0,written conflict fabulous,2.991167,7.106799,1443,red
135,ar\vr,0,Кластер 0,reservations linking,10.195602,12.259496,751,red
134,ar\vr,0,Кластер 0,interfaces neutral,10.443533,13.809915,586,red
136,ar\vr,0,Кластер 0,committees parallel,6.735260,3.613983,173,red
137,ar\vr,1,Кластер 1,postcards looked republic detector,10.474474,6.220012,1397,green
139,ar\vr,1,Кластер 1,michael tobacco,6.210880,12.721264,1007,green
138,ar\vr,1,Кластер 1,filling volunteers academics,10.833920,8.652737,773,green
140,ar\vr,1,Кластер 1,celtic automation,12.197275,4.723751,715,green
143,ar\vr,2,Кластер 2,enquiries implementation,3.580718,8.089951,1249,blue
142,ar\vr,2,Кластер 2,previously standing languages commands,14.668089,8.467136,889,blue


In [30]:
#сохраним csv файл
sort_df.to_csv('hse_test_out.csv',index=False)

# Часть 2

In [31]:
#создадим копию нашего датафрейма
cp_df = deepcopy(sort_df)

In [32]:
#функция для разделения длиннных строк
def new_string(string):
    return '<br>'.join(textwrap.wrap(string, width=20))

In [33]:
cp_df['keyword'] = cp_df['keyword'].apply(new_string)

In [34]:
cp_df

,area,cluster,cluster_name,keyword,x,y,count,color
133,ar\vr,0,Кластер 0,written conflict<br>fabulous,2.991167,7.106799,1443,red
135,ar\vr,0,Кластер 0,reservations linking,10.195602,12.259496,751,red
134,ar\vr,0,Кластер 0,interfaces neutral,10.443533,13.809915,586,red
136,ar\vr,0,Кластер 0,committees parallel,6.735260,3.613983,173,red
137,ar\vr,1,Кластер 1,postcards looked<br>republic detector,10.474474,6.220012,1397,green
...,...,...,...,...,...,...,...,...
127,worlds,2,Кластер 2,ringtone parental,11.723895,4.363994,471,blue
129,worlds,2,Кластер 2,recipient traffic,5.593629,0.553368,236,blue
132,worlds,3,Кластер 3,immunology plates,2.407028,7.651527,1653,orange
130,worlds,3,Кластер 3,holdings herbal,3.986508,10.906340,1476,orange


In [35]:
if not os.path.exists("images"):
        os.mkdir("images")

In [36]:
for area in sort_areas:
    query = cp_df[cp_df['area']==area][['cluster_name','x', 'y','keyword']]
    fig = px.scatter(query, x='x', y='y', color='cluster_name', 
                     color_discrete_map={
                                        'Кластер 0': 'red',
                                        'Кластер 1': 'green',
                                        'Кластер 2': 'blue',
                                        'Кластер 3': 'orange'},
                     text='keyword'
                    )
                     
    fig.update_traces(textposition="bottom center",
                      marker=dict(size=36,
                                  line=dict(width=2,
                                            color='DarkSlateGrey')),
                      textfont=dict(
                                   family="sans-serif",
                                   size=26,
                                   color="black"
                                   )
                     )
                     
    fig.update_layout(
                      xaxis = dict(tickfont=dict(size=30)),
                      yaxis = dict(tickfont=dict(size=30)),
                      plot_bgcolor='rgba(0,0,0,0)',
                      legend=dict(bordercolor="Black",
                                  borderwidth=2,
                                  font=dict(size=30)),
                      height=1200,
                      width=2000,
                      title=dict(text= f'Area: {area}',
                                 xanchor= 'left',
                                 yanchor= 'middle',
                                 font=dict(size=60)
                                 ),
                      font=dict(size=30),
                      margin=dict(l=120, r=120, t=120, b=120)
                     )
    
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', ticks="outside", ticklen=10)
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', ticks="outside", ticklen=10)

    fig.add_annotation(dict(
                            font=dict(color='black',size=27),
                            x=0.87,
                            y=-0.078,
                            showarrow=False,
                            text="Источник: система интеллектуального анализа больших данных iFORA",
                            textangle=0,
                            xanchor='center',
                            xref="paper",
                            yref="paper"
                           )
                      )

    pio.write_image(fig, f'images/fig_{area}.png', width=2500, height=1800)